# **This code extracts data from portfolio file (either in .pdf or .xlsx  format) and latest trading data from Dhaka Stock Exchange Website, then portrays the current position of the portfolio. The scenario is updated every 30 seconds for the sake of continuous monitoring. **





**Disclaimer**

    1. You need to upload the portfolio file from the "Files" section. The accepted format is either .pdf or .xlsx. 
    2. The broker provided .pdf file might not suit the code, in that case you may provide the portfolio data in .xlsx format. Please ensure that the .xlsx file has TRADING.CODE, Total Quantity, Average Cost and Total Cost in the first four columns. 
    3. The gain/loss is calculated considering Latest Trading Price during the trading hours and Closing Price after trading hours.


In [1]:
# take care of the warnings
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter('ignore')

# check if bdshare pypdf and tabula are installed if not install
try:
    from bdshare import get_current_trade_data
except:
    !pip install bdshare
    from bdshare import get_current_trade_data

try:
    import tabula
except:
    !pip install tabula-py
    import tabula

# import required modules
import pandas as pd
import numpy as np

# set the number format
pd.options.display.float_format = "{:.2f}".format

# set pandas display width
pd.set_option("display.max_rows", 50)
pd.set_option("display.max_columns", 14)
pd.set_option("display.width", 1000)

# function for extracting the data
def dse_current(filename):

    # get the data
    if ".pdf" in filename:
        df_org = tabula.read_pdf(filename, multiple_tables=True, pages="all")[0]
        port = df_org.iloc[1:-2, :]
        req_cols = ["Company", "Total", "Average", "Total Cost"]
        port = port[req_cols]
        port["Total Cost"] = port["Total Cost"].str.replace(",", "")
        cols = ["Total", "Average", "Total Cost"]
        port[cols] = port[cols].apply(pd.to_numeric, errors="coerce")
        port.columns = ["TRADING.CODE", "Total Quantity", "Average Cost", "Total Cost"]

    elif ".xlsx" in filename:
        port = pd.read_excel(filename)
        port.columns = ["TRADING.CODE", "Total Quantity", "Average Cost", "Total Cost"]

    # Get the instruments under portfolio
    instruments = port["TRADING.CODE"]
    instruments = instruments.tolist()

    # create the blank file for merging all instruments' data
    df = []

    # extracts all instruments data
    for instruments in instruments:
        stock_data = get_current_trade_data(instruments)
        df.append(stock_data)

    df = pd.concat(df)
    df = df.sort_values(by=["symbol"])
    df = df[
        ["symbol", "ltp", "high", "low", "close", "ycp", "trade", "value", "volume"]
    ]
    df = df.reset_index()
    df = df.drop(columns=["index"])
    df.columns = [
        "TRADING.CODE",
        "LTP",
        "HIGH",
        "LOW",
        "CLOSEP",
        "YCP",
        "TRADE",
        "VALUE",
        "VOLUME",
    ]
    cols = df.columns.drop(["TRADING.CODE"])
    df[cols] = df[cols].apply(pd.to_numeric, errors="coerce")

    # merge the data with uploaded portfolio
    df = pd.merge(port, df, on="TRADING.CODE")

    # get current gain/loss
    if df["CLOSEP"].sum() == 0:
        df["Current_Val"] = np.where(
            df["LTP"] > 0,
            df["Total Quantity"] * df["LTP"],
            df["Total Quantity"] * df["YCP"],
        )
    else:
        df["Current_Val"] = df["Total Quantity"] * df["CLOSEP"]

    df["Gain/Loss"] = df["Current_Val"] - df["Total Cost"]

    # generate the summary table
    total_quan = df["Total Quantity"].sum()
    total_cost = df["Total Cost"].sum()
    mkt_val = df["Current_Val"].sum()
    gain = df["Gain/Loss"].sum()
    summary = [total_quan, total_cost, mkt_val, gain]
    summary_tab = pd.DataFrame(
        np.array(summary).reshape(1, 4),
        columns=["Total Shares", "Total Cost", "Total Market Value", "Total Gain/Loss"],
    )

    return (df, summary_tab)

# Set the time format
from datetime import datetime
import time
from IPython.display import clear_output
from pytz import timezone

bd = timezone("Asia/Dhaka")

# function for repeating the previous function after certain interval
def periodic_position(interval, filename):
    while True:
        current_pos = dse_current(filename)[0]
        summary_tab = dse_current(filename)[1]
        clear_output(wait=True)
        print("As of:")
        print(datetime.now(bd).strftime("%H:%M:%S.%f - %b %d %Y"))
        print("---")
        print(current_pos)
        print("---")
        # print("Current Total Gain/Loss is: {:2f}".format(gain))
        print(summary_tab)
        print("---")
        time.sleep(interval)

In [ ]:
# now keep producing the current status of the portfolio
periodic_position(30, "P 05-10-2020.pdf")

As of:
22:15:17.987960 - Oct 05 2020
---
  TRADING.CODE  Total Quantity  Average Cost  Total Cost    LTP   HIGH    LOW  CLOSEP    YCP  TRADE  VALUE   VOLUME  Current_Val   Gain/Loss
0     BANKASIA          800405         18.60 14889729.66  19.00  19.10  18.80   18.90  18.80    170   8.95   473214  15127654.50   237924.84
1          DBH           25000         94.29  2357227.50  92.60  92.60  92.60   92.60  92.60     53   4.05    43702   2315000.00   -42227.50
2          EBL          191080         36.82  7034935.88  36.60  37.00  36.00   36.30  36.80     38   3.16    86455   6936204.00   -98731.88
3   JAMUNABANK         3017266         17.98 54247333.93  17.70  17.90  17.40   17.80  17.70    146  10.57   597270  53707334.80  -539999.13
4   MERCANBANK         2542642         12.94 32909199.86  12.90  13.00  12.60   12.80  12.70    295  19.02  1489641  32545817.60  -363382.26
5      NCCBANK         4413769         13.96 61627468.32  13.30  13.60  13.30   13.30  13.50    168   6.44   4829